# Integrate app into UI interface: Gradio

- **input fields**: 
    - text box for user to describe current game information
        - optional: voice-to-text capability
    - optional: dedicated input fields for current game information (pitcher name, batter name, who is on base, fielding alignment, etc)
- **output fields**:
    - LLM Chat with history
    - optional: toggle on/off history visibility

In [43]:
# import all dependencies and libraries
from dotenv import load_dotenv
import os
from langchain import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
# from transformers import pipeline
import gradio as gr


# Load environment variables.
load_dotenv()
# Set the model name for our LLMs.
GEMINI_MODEL = "gemini-1.5-flash"
# Store the API key in a variable.
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")


# instatiate the LLM
llm = ChatGoogleGenerativeAI(google_api_key=GEMINI_API_KEY, model=GEMINI_MODEL, temperature=0.5)


# create a template for the chatbot personality
template = """
You are a baseball coach. Answer only questions that would pertaining to baseball.
If the human asks questions not related to baseball, remind them that your job is to help
them learn answer baseball questions, and ask them for a question on that topic. If they ask a question which
there is not enough information to answer, tell them you don't know and don't make up an 
answer.

Current conversation:
{history}
Human: {input}
AI Assistant:"""

prompt = PromptTemplate(
    input_variables=["history", "input"],
    template=template
)


# create conversation chain with memory
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(llm=llm, memory=memory, prompt=prompt,verbose=True)

In [44]:
# function for quick tests:

def placeholder_fn(query, history):
    # add user's message to conversation history
    history.append(("User", query))
    # get response from AI
    result = conversation.predict(input=query)
    # add AI response to conversation history
    history.append(("AI", result))
    # format conversation history for display
    formatted_history = "\n".join([f"{sender}: {msg}" for sender, msg in history])

    return formatted_history, history

## stretch goal: voice to text functionality
add this functionality after core functionality is complete.

In [ ]:
# # add voice-to-text input
# import torch
# from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
# from datasets import load_dataset


# device = "cuda:0" if torch.cuda.is_available() else "cpu"
# torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# model_id = "openai/whisper-large-v3"

# model = AutoModelForSpeechSeq2Seq.from_pretrained(
#     model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
# )
# model.to(device)

# processor = AutoProcessor.from_pretrained(model_id)

# pipe = pipeline(
#     "automatic-speech-recognition",
#     model=model,
#     tokenizer=processor.tokenizer,
#     feature_extractor=processor.feature_extractor,
#     torch_dtype=torch_dtype,
#     device=device,
# )

# dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
# sample = dataset[0]["audio"]

# result = pipe(sample)
# print(result["text"])

In [ ]:
# def transcribe(audio):
#     if audio is not None: 
#         result = transcriber(audio)
#         return result['text']
#     else: 
#         return ""

# def process_input(audio, text, history):
#     message = transcribe(audio) if audio is not None else text
#     return placeholder_fn(message, history)

## Integrate LLM/Transformer function(s) into Gradio

### Gradio: using default format

In [49]:
# gradio app

chat_app = gr.Interface(
    fn = placeholder_fn, 
    inputs = [
        # gr.Audio(source="microphone", type="filepath", label="Speak your message (optional)"),
        gr.Textbox(lines = 2, label = "Or type your message here"),
        gr.State([])
    ],
    outputs = [
        gr.Textbox(lines = 10, label="Coach AI response:", show_copy_button=True),
        gr.State()
    ],
    title="Coach AI",
    description="Chat with an AI using your voice or by typing. The AI remembers your conversation history."
)

c:\Users\elcoo\anaconda3\Lib\site-packages\gradio\interface.py:393: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


In [50]:
# launch app locally

chat_app.launch()

* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


### Gradio: Using Blocks for formatting

#### Theme Builder

In [51]:
gr.themes.builder()

c:\Users\elcoo\anaconda3\Lib\site-packages\gradio\components\chatbot.py:222: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


#### App with applied theming

In [84]:
import gradio as gr

theme = gr.themes.Ocean(
    primary_hue="indigo",
    secondary_hue="fuchsia",
)

with gr.Blocks(theme=theme) as chat_app:
    # Add an image to the top center
    gr.Image("images\colorado-rockies.png", show_label=False, container=False, height=200)
    gr.HTML("""
    <style>
    .center-text { text-align: center; }
    </style>
            """)
     

    gr.HTML('<h1 class="center-text">Moneyball Part Deux, part 3</h1>')
    gr.HTML('<p class="center-text">Chat with your personal AI Coach about anything related to baseball.</p>')
    
    chatbot = gr.Chatbot()
    chat_input = gr.MultimodalTextbox(
        interactive=True, 
        placeholder="Enter message or upload file...", 
        show_label=False,
    )

    clear = gr.Button("Clear", variant="primary")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        bot_message, updated_history = placeholder_fn(history[-1][0], history[:-1])
        history[-1][1] = bot_message
        return history

    chat_input.submit(user, [chat_input, chatbot], [chat_input, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)

if __name__ == "__main__":
    chat_app.launch()


c:\Users\elcoo\anaconda3\Lib\site-packages\gradio\components\chatbot.py:222: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7897

To create a public link, set `share=True` in `launch()`.
